## Introduction

## Setup

### Configure API keys

To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
# Print the current working directory
import os
HOME = os.getcwd()
print(HOME)

## Install YOLO11 via Ultralytics

In [ ]:
%pip install -q --upgrade ultralytics supervision roboflow
import ultralytics

# Check the installation
ultralytics.checks()

## Fine-tune YOLO11 on custom dataset

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/liangdianzhong/-qvdww) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from kaggle_secrets import UserSecretsClient
from roboflow import Roboflow

# Get the Roboflow API key
roboflow_secret_value = UserSecretsClient().get_secret("ROBOFLOW_SECRET_KEY")
rf = Roboflow(api_key=roboflow_secret_value)

# Download the dataset
project = rf.workspace("ben10-zvdpq").project("eye-retinal-disease")
version = project.version(1)
dataset = version.download("yolov11")              

### Custom Training

In [ ]:
%cd {HOME}

# Train the model using the downloaded dataset and the YOLO11 medium model with other parameters
!yolo task=detect mode=train model=yolo11m.pt data={dataset.location}/data.yaml epochs=50 imgsz=640 plots=True

NOTE: The results of the completed training are saved in {HOME}/runs/detect/train/. Let's examine them.



In [ ]:
# Check the training results
!ls {HOME}/runs/detect/train/

In [ ]:
from IPython.display import Image as IPyImage

# Display the confusion matrix plot
IPyImage(filename=f"{HOME}/runs/detect/train/confusion_matrix.png", width=600)

In [ ]:
# Display the training results plot
IPyImage(filename=f"{HOME}/runs/detect/train/results.png", width=600)

In [ ]:
# Display the validation batched prediction plot
IPyImage(filename=f"{HOME}/runs/detect/train/val_batch0_pred.jpg", width=600)

### Validate fine-tuned model

In [ ]:
# Evaluate the model using the validation dataset and the best weights
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

### Inference with custom model

In [ ]:
# Predict the test dataset using the best weights
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

NOTE: Let's take a look at few results.



In [ ]:
import glob
from IPython.display import Image as IPyImage, display

# Get the latest prediction folder
latest_folder = max(glob.glob(f"{HOME}/runs/detect/predict*/"), key=os.path.getmtime)

# Display the first 3 images from the test dataset
for img in glob.glob(f"{latest_folder}/*.jpg")[:3]:
    display(IPyImage(filename=img, width=600))
    print("\n")

## Conclusion